In [1]:
import os
import pandas as pd
import json
from pandas import json_normalize
from skimpy import skim
import numpy as np
import plotly_express as px
import matplotlib.pylab as plt
pd.set_option('display.float_format', lambda x: '{:.0f}'.format(x))

In [2]:
columns = [
    'categories', "manufacturer", "title", "lastUpdate", "lastPriceChange",
    "rootCategory", "productType", "parentAsin", "variationCSV", "asin",
    "type", "hasReviews", "trackingSince", "brand", "productGroup", "partNumber",
    "model", "color", "edition", "isAdultProduct", "isEligibleForTradeIn",
    "isEligibleForSuperSaverShipping", "offers", "buyBoxSellerIdHistory",
    "isRedirectASIN", "isSNS", "author", "binding", "numberOfItems",
    "numberOfPages", "publicationDate", "releaseDate", "languages",
    "lastRatingUpdate", "size", "frequentlyBoughtTogether", "promotions",
    "coupon", "categoryTree"
]

path_to_json_files = '../out/'
df_list = []

for filename in os.listdir(path_to_json_files):
    if filename.endswith('.json'):
        with open(os.path.join(path_to_json_files, filename)) as user_file:
            parsed_json = json.load(user_file)
        if 'products' in parsed_json:
            df = json_normalize(parsed_json['products'])
            df1 = df[columns]
            df_list.append(df1)

combined_df = pd.concat(df_list, ignore_index=True)

# combined_df.to_csv('result_8000.csv')


In [3]:
columns2 = [
    'categories', "lastUpdate", "lastPriceChange",
    "rootCategory", "productType", "parentAsin", "asin",
    "type", "hasReviews", "trackingSince", "brand", "productGroup", "partNumber",
    "edition", "isAdultProduct", "isEligibleForTradeIn",
    "isEligibleForSuperSaverShipping", "offers", "buyBoxSellerIdHistory",
    "isRedirectASIN", "isSNS", "author", "binding", "publicationDate", "releaseDate", "languages",
    "frequentlyBoughtTogether", "promotions", "coupon", "categoryTree"
]

test = combined_df[columns2]


In [33]:
category = test[['productGroup', 'asin', 'categories', 'categoryTree', "rootCategory"]]
filters = pd.read_csv('home & kitchen.csv')
filters.columns=['nodeID', 'nodePath', 'Query', 'Refinement Link', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6']

nodid = list(filters['nodeID'])
nodepath = list(filters['nodePath'])

cats = []
for i, ids in enumerate(nodid):
    for j in range(len(category['categories'])):
        if ids in category['categories'][j]:
            cats.append((nodepath[i], ids, category['asin'][j]))

cats_df = pd.DataFrame(cats, columns=['nodepath', 'nodeID', 'asin'])
result=category.merge(cats_df, on = 'asin', how = 'left')
result.to_csv('filter_merge_categories.csv')

In [ ]:
plot_df = result.groupby(['nodeID', 'nodepath'], as_index=False).agg({'asin': 'count'})
check = result.groupby(['asin', 'nodeID', 'nodepath'], as_index=False).count()
chek_s = check['nodepath'].str.split('/', expand=True)
ch = pd.concat([check, chek_s], axis=1)
ch['lastCategory'] = ch['nodepath'].str.split('/').str[-1]
check1 = ch.drop_duplicates(subset='asin', keep='first')
check1.columns = ['asin', 'nodeID', 'nodepath', 'productGroup',
                  'categories', 'categoryTree', 'rootCategory', 'path0',
                  'path1', 'path2', 'path3', 'path4', 'path5',
                  'path6', 'path7', 'path8', 'lastCategory']

check1 = check1[['asin', 'nodeID', 'nodepath', 'path0', 'path1', 'path2',
                 'path3', 'path4', 'path5', 'path6', 'path7', 'path8', 'lastCategory']]

check1.to_csv('asin_path_id.csv', index=False)

In [67]:
df=check1.groupby('path1', as_index=False).count()[['path1','asin']]

fig = px.pie(df, values='asin', names='path1')
fig.show()

fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  )
fig.show()

In [77]:

fig = px.treemap(check1.groupby(['path0', 'path1', 'path2', 'path3',
       'path4', 'path5', 'path6'], as_index=False).count(),
                 path=['path0', 'path1', 'path2', 'path3',
       'path4', 'path5', 'path6'],
                 values='asin',
                 title="Categories Treemap",
                 width=800,
                 height=800,
                 color='path3')
fig.update_traces(textinfo="label+percent parent")
chart_data = fig.data[0].hovertext

fig.show()
fig.write_html("categories_treemap2.html")


In [78]:
ch_group=ch[[0,1,2,3,4,5,'categories']].groupby([0,1,2,3,4,5],as_index = False).sum()

fig = px.sunburst(ch_group,
                  path=[0,1,2,3,4,5,],
                  values='categories',
                  title="Tree cats Distribution",
                  width=800, 
                  height=800,
                  color=3)
fig.update_traces(textinfo="label+percent parent")
fig.show()
fig.write_html("ch.html")